# Juliaによる非線形方程式の全解探索アルゴリズムの実装

## Krawczyk法

解きたい方程式を$ f(x)=0, f:R^n \rightarrow R^n$とする。

解の存在を検証したい領域を$I \in R^n$とする。

cを$I$の中心とする。

$R$を$f'(c)$の近似逆行列とする。

$K(I):=c-Rf(c)+(R-Rf'(I)(I-c))$をKrawczyk写像という。

$K(I) \in int(I)$が成立するならば$f(x)=0$の真の解が$I$内に一意に存在する。


## 解の非存在条件
Krawczyk写像を利用し、解の非存在性を示すことができる。区間$X$に対し、$X \setminus K(X)$には解が存在しないので、
与えられた区間ベクトル$X \in IR^n $に対して、

$K(X) \cap X = \emptyset$

ならば、非線形方程式$f(X) = 0$の解は存在しない。

また、区間演算の性質より、以下のような非存在条件が考えられる。

与えられた区間ベクトル$X \in IR^n$に対して

$f(X) \notin 0$

または

$f(c) + f'(X)(X-c) \notin 0$

ならば、非線形方程式$f(X) = 0$の解は存在しない。

## Juliaによる区間演算

In [171]:
using IntervalArithmetic

X = (0.1 .. 0.2)
Y = (0.3 .. 0.4)

@show X+Y
@show X-Y
@show X*Y
@show X/Y

X + Y = [0.399999, 0.600001]
X - Y = [-0.300001, -0.0999999]
X * Y = [0.0299999, 0.0800001]
X / Y = [0.249999, 0.666667]


[0.249999, 0.666667]

## Juliaによる線形代数演算

In [172]:
using LinearAlgebra

X = [1 2;3 4]
Y = [5 6;7 8]

@show typeof(X)
@show X+Y
@show X*Y
@show inv(X) 

typeof(X) = Matrix{Int64}
X + Y = [6 8; 10 12]
X * Y = [19 22; 43 50]
inv(X) = [-1.9999999999999996 0.9999999999999998; 1.4999999999999998 -0.4999999999999999]


2×2 Matrix{Float64}:
 -2.0   1.0
  1.5  -0.5

## Juliaによる自動微分

In [173]:
using ForwardDiff

f((x1,x2)) = x1^2-x2-1
g((x1,x2)) = (x1-2)^2 -x2-1
F((x,y))=[f((x,y));g((x,y))]
j1 = ForwardDiff.jacobian(F,[1. 1.])
j2 = ForwardDiff.jacobian(F,[(0. ..  1.) (0. .. 1.)])

@show j1
@show j2

j1 = [2.0 -1.0; -2.0 -1.0]
j2 = Interval{Float64}[[0, 2] [-1, -1]; [-4, -2] [-1, -1]]


2×2 Matrix{Interval{Float64}}:
   [0, 2]  [-1, -1]
 [-4, -2]  [-1, -1]

## 全解探索アルゴリズムのソースコード

In [174]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

function orig_intersect(X,Y)
    L = X*1
    for i = 1:size(X)[1]
        L[i]= intersect(X[i],Y[i])
    end
    return L
end

function orig_is_empty(X,Y)
    for i = 1:size(X)[1]
        if ( isempty(intersect(X[i],Y[i])) )
            return true
        end
    end
    return false
end

function orig_devide(X)
    #最も長い辺を二等分する
    max_diam=0
    max_index=0
    min_diam=diam(X[1])
    min_index=1
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
        if diam(x)<min_diam
            min_diam=diam(x)
            min_index=i
        end
    end

    if diam(X[min_index])<10^(-8)
        return false
    end
    
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    return X1,X2
end

function border(K,I)
    c=0.9
    K_width=diam.(K)
    I_width=diam.(I)
    max=0
    for i=1:size(K)[1]
        if (K_width[i]/I_width[i])>max
            max=K_width[i]/I_width[i]
        end
    end
    if max<=0.9
        return true
    else
        return false
    end
end

function remove_duplication(S)
    tmp_index=[]
    new_s=[]
    for i=1:size(S)[1]-1
        for j = i+1:size(S)[1]
            if !orig_is_empty(S[i],S[j])
                
                if (i in tmp_index) || (j in tmp_index)
                    push!(tmp_index,i)
                    push!(tmp_index,j)
                    continue
                end
                push!(tmp_index,i)
                push!(tmp_index,j)
                I = orig_intersect(S[i],S[j])
                push!(new_s,I)
            end
        end
    end
    for i=1:size(S)[1]
        if !(i in tmp_index)
            push!(new_s,S[i])
        end
    end
    
    return new_s
end


function allsol(F,X)
    dimention = size(X)[1]
    
    L = [X] #調査する区間のリスト
    S = Array[] #見つかった解のリスト
    
    while (size(L)[1] != 0)
        
        X = popfirst!(L) #Lから区間を一つ取り出す
        
        #非存在判定
        if !( orig_is_subset([(0. .. 0.) for i=1:dimention], F(X)) ) 
            continue
        end
        
        c = mid.(X)
        j = ForwardDiff.jacobian(F,X)
        
        #非存在判定
        if !( orig_is_subset([(0. .. 0.) for i=1:dimention], F(c)+j*(X-c)) )
            continue
        end
        
        R = []
        #逆行列計算
        try
            R = inv(ForwardDiff.jacobian(F,c))
        catch e
            #逆行列計算に失敗したら区間を分割
            try
                X1,X2 = orig_devide(X)
                push!(L, X1)
                push!(L, X2)
            catch 
                continue
            end
            continue
        end
        
        M = Matrix{Float64}(I,size(R))-R*j
        K = c -R*F(c)+M*(X-c)
        
        #解なし
        if (orig_is_empty(K,X))
            continue 
        end
        
        #解あり
        if (orig_is_subset(K,X))　
            push!(S, K) #解のリストに追加
            continue
        end
        
        # 境界に乗ってた時
        if border(K,X)
            push!(L,K)
            continue
        end
        
        #分割して候補者区間に追加
        try
            X1,X2 = orig_devide(orig_intersect(K,X))
            #K and X を2分割
            push!(L, X1)
            push!(L, X2)
            continue
        catch
            continue
        end
    end
    
    #重複を排除
    S = remove_duplication(S)
    
    #区間を狭める
    for i =1:size(S)[1]
        si=S[i]
        while(maximum(radius,si) >= 10^-5)
            j=0
            try
                j = ForwardDiff.jacobian(F,si)
            catch
                break
            end
            c = mid.(si)
            R=0
            try 
                R = inv(ForwardDiff.jacobian(F,c)) 
            catch
                break
            end
            M = Matrix{Float64}(I,size(R))-R*j
            k = c -R*F(c)+M*(si-c)
            if orig_is_subset(k,si)
                si=k
            else
                break
            end
        end
        S[i]=si
    end
    
    for s=1:size(S)[1] 
        print("解");print(s);print(": ");
        for i in S[s]
            print(i)
            print(" ")
        end
        println("")
    end
    
    return S
end

# 重解　＝＞区間幅に制限をつける
# 高速化？

allsol (generic function with 1 method)

## 検証1

In [175]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

f((x1, x2)) = 2*x1^3 + 2*x1*x2 - 22*x1+ x2^2 + 10 + pi
g((x1, x2)) = x1^2 + 2*x1*sin(x2) + 2*x2^3 - 14*x2 + 9.1

F1((x,y))=[f((x,y));g((x,y))]
answer=allsol(F1,X)
@show answer

解1: [-4.14462, -4.14461] [-3.28814, -3.28813] 
解2: [3.2574, 3.25741] [-3.18413, -3.18412] 
解3: [0.820019, 0.820025] [-2.93392, -2.93389] 
解4: [1.08436, 1.08437] [1.97179, 1.9718] 
解5: [-3.4131, -3.41309] [1.69858, 1.69859] 
解6: [0.720303, 0.720304] [0.853378, 0.853379] 
解7: [-3.43452, -3.43451] [1.40441, 1.40442] 
answer = Any[Interval{Float64}[[-4.14462, -4.14461], [-3.28814, -3.28813]], Interval{Float64}[[3.2574, 3.25741], [-3.18413, -3.18412]], Interval{Float64}[[0.820019, 0.820025], [-2.93392, -2.93389]], Interval{Float64}[[1.08436, 1.08437], [1.97179, 1.9718]], Interval{Float64}[[-3.4131, -3.41309], [1.69858, 1.69859]], Interval{Float64}[[0.720303, 0.720304], [0.853378, 0.853379]], Interval{Float64}[[-3.43452, -3.43451], [1.40441, 1.40442]]]


7-element Vector{Any}:
 Interval{Float64}[[-4.14462, -4.14461], [-3.28814, -3.28813]]
 Interval{Float64}[[3.2574, 3.25741], [-3.18413, -3.18412]]
 Interval{Float64}[[0.820019, 0.820025], [-2.93392, -2.93389]]
 Interval{Float64}[[1.08436, 1.08437], [1.97179, 1.9718]]
 Interval{Float64}[[-3.4131, -3.41309], [1.69858, 1.69859]]
 Interval{Float64}[[0.720303, 0.720304], [0.853378, 0.853379]]
 Interval{Float64}[[-3.43452, -3.43451], [1.40441, 1.40442]]

### kvのallsolによる計算結果

[-4.1446150444051498,-4.1446150444051452]	[-3.28813192969558,-3.2881319296955777]

[0.82002197462817749,0.82002197462817839]	[-2.9339051715639717,-2.933905171563969]

[-3.4130997082062136,-3.4130997082062104]	[1.6985839371679012,1.6985839371679068]


[-3.4345149364856549,-3.4345149364856526]	[1.4044134733988069,1.4044134733988121]

[0.72030308132508302,0.7203030813250837]	[0.85337813273684248,0.85337813273684371]

[1.0843602587813725,1.0843602587813746]	[1.971798770579664,1.9717987705796661]

## 検証２

In [176]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

f((x1,x2)) = x1 + 2*x2 +1
g((x1,x2)) = x1^2 + 2*(x2^2) -1

F2((x,y))=[f((x,y));g((x,y))]
answer=allsol(F2,X)
@show answer

解1: [-1.00001, -0.999999] [-3.985e-09, 3.985e-09] 
解2: [0.333333, 0.333334] [-0.666667, -0.666666] 
answer = Any[Interval{Float64}[[-1.00001, -0.999999], [-3.985e-09, 3.985e-09]], Interval{Float64}[[0.333333, 0.333334], [-0.666667, -0.666666]]]


2-element Vector{Any}:
 Interval{Float64}[[-1.00001, -0.999999], [-3.985e-09, 3.985e-09]]
 Interval{Float64}[[0.333333, 0.333334], [-0.666667, -0.666666]]

### kvのallsolによる計算結果

[0.3333333333333332,0.33333333333333349]	[-0.66666666666666686,-0.66666666666666651]

[-1.0000000000000005,-0.99999999999999977]	[-5.6222596042849915e-17,2.7044130804046339e-17]

## 検証3

In [177]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

f((x1, x2)) = x1^2 + x2^2 - 1
g((x1, x2)) = x1^2 - x2^4

F3((x,y))=[f((x,y));g((x,y))]
answer=allsol(F3,X)
@show answer

解1: [-0.618035, -0.618033] [-0.786152, -0.786151] 
解2: [-0.618035, -0.618033] [0.786151, 0.786152] 
解3: [0.618033, 0.618035] [-0.786152, -0.786151] 
解4: [0.618033, 0.618035] [0.786151, 0.786152] 
answer = Any[Interval{Float64}[[-0.618035, -0.618033], [-0.786152, -0.786151]], Interval{Float64}[[-0.618035, -0.618033], [0.786151, 0.786152]], Interval{Float64}[[0.618033, 0.618035], [-0.786152, -0.786151]], Interval{Float64}[[0.618033, 0.618035], [0.786151, 0.786152]]]


4-element Vector{Any}:
 Interval{Float64}[[-0.618035, -0.618033], [-0.786152, -0.786151]]
 Interval{Float64}[[-0.618035, -0.618033], [0.786151, 0.786152]]
 Interval{Float64}[[0.618033, 0.618035], [-0.786152, -0.786151]]
 Interval{Float64}[[0.618033, 0.618035], [0.786151, 0.786152]]

### kvのallsolによる計算結果

[-0.61803398874989513,-0.61803398874989456]	[-0.7861513777574235,-0.78615137775742305]

[-0.61803398874989513,-0.61803398874989456]	[0.78615137775742305,0.7861513777574235]

[0.61803398874989456,0.61803398874989513]	[-0.7861513777574235,-0.78615137775742305]

[0.61803398874989456,0.61803398874989513]	[0.78615137775742305,0.7861513777574235]

## 検証4(重解)

In [181]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

f((x1, x2)) = x1^2 + x2^2 - 1
g((x1, x2)) = 2*x1^2 - x2 - 1

F4((x,y))=[f((x,y));g((x,y))]
answer=allsol(F4,X)
@show answer

F4(X) = Interval{Float64}[[-1, 2e+06], [-1001, 2.001e+06]]
解1: [-0.866026, -0.866024] [0.499998, 0.500002] 
解2: [0.866024, 0.866026] [0.499998, 0.500002] 
answer = Any[Interval{Float64}[[-0.866026, -0.866024], [0.499998, 0.500002]], Interval{Float64}[[0.866024, 0.866026], [0.499998, 0.500002]]]


2-element Vector{Any}:
 Interval{Float64}[[-0.866026, -0.866024], [0.499998, 0.500002]]
 Interval{Float64}[[0.866024, 0.866026], [0.499998, 0.500002]]

### kvのallsolによる計算結果

[-0.86602540378443882,-0.86602540378443848]	[0.49999999999999983,0.50000000000000023]

[0.86602540378443848,0.86602540378443882]	[0.49999999999999983,0.50000000000000023]


## 検証5 3次元以上

In [185]:
X = [(-1000. ..1000.),(-1000. .. 1000.),(-1000. .. 1000.)]

f((x1, x2, x3)) = 2*x1 + x2^2 + x3
g((x1, x2, x3)) = 2*x1^2 - x2 - x3
h((x1, x2, x3)) = x1 + x2 + x3

F5((x,y,z))=[f((x,y,z));g((x,y,z));h((x,y,z))]

answer=allsol(F5,X)
@show answer

解1: [-6.56039e-20, 6.52403e-20] [-7.71662e-08, 7.71617e-08] [-7.71617e-08, 7.71662e-08] 
解2: [5.11059e-17, 5.15905e-17] [0.999997, 1.00001] [-1.00001, -0.999997] 
解3: [-0.500001, -0.499999] [-0.366026, -0.366024] [0.866024, 0.866027] 
解4: [-0.500001, -0.499999] [1.36602, 1.36603] [-0.866026, -0.866025] 
answer = Any[Interval{Float64}[[-6.56039e-20, 6.52403e-20], [-7.71662e-08, 7.71617e-08], [-7.71617e-08, 7.71662e-08]], Interval{Float64}[[5.11059e-17, 5.15905e-17], [0.999997, 1.00001], [-1.00001, -0.999997]], Interval{Float64}[[-0.500001, -0.499999], [-0.366026, -0.366024], [0.866024, 0.866027]], Interval{Float64}[[-0.500001, -0.499999], [1.36602, 1.36603], [-0.866026, -0.866025]]]


4-element Vector{Any}:
 Interval{Float64}[[-6.56039e-20, 6.52403e-20], [-7.71662e-08, 7.71617e-08], [-7.71617e-08, 7.71662e-08]]
 Interval{Float64}[[5.11059e-17, 5.15905e-17], [0.999997, 1.00001], [-1.00001, -0.999997]]
 Interval{Float64}[[-0.500001, -0.499999], [-0.366026, -0.366024], [0.866024, 0.866027]]
 Interval{Float64}[[-0.500001, -0.499999], [1.36602, 1.36603], [-0.866026, -0.866025]]

[-4.9406564584124655e-324,4.9406564584124655e-324]
[-9.8813129168249309e-324,9.8813129168249309e-324]
[-9.8813129168249309e-324,9.8813129168249309e-324]

[-1.9075222407249442e-16,3.1292380856700906e-17]
[0.99999999999999966,1.0000000000000007]
[-1.0000000000000005,-0.99999999999999977]

[-0.50000000000000023,-0.49999999999999994]
[-0.36602540378443877,-0.36602540378443854]
[0.86602540378443848,0.86602540378443882]

[-0.50000000000000023,-0.49999999999999994]
[1.3660254037844383,1.3660254037844391]
[-0.86602540378443893,-0.86602540378443848]


## 今後

- 高速化
- KVのallsol_simpleとの速度比較